In [ ]:
import sys

sys.path.insert(0, 'backend_functions')

import selection_functions as sf

import importlib

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb


import awkward
import matplotlib.pyplot as plt
import pandas as pd

import ROOT

import top 
from top import *

import uncertainty_functions 
from uncertainty_functions import *

import xsec_functions 
from xsec_functions import smear_matrix

from ROOT import TH1D, TH2D, TDirectory, TH1F, TH2F

from selection_functions import *

import json

In [ ]:
xvar = 'shr_energy_cali'
true_var = 'elec_e'
bins = [0.02, 0.22, 0.42, 0.62, 0.82 , 1.22,  7]

xlow = bins[0]
xhigh = 2.5

x_label = "Shower Energy [GeV]"

x_ticks = [0.02, 0.22, 0.42, 0.62, 0.82 , 1.22, 2.5]

bincenters = 0.5*(np.array(x_ticks)[1:]+np.array(x_ticks)[:-1])

In [ ]:
with open('variations/FHCVariations_July15.json') as f_fhc:
    fhc_dict = json.load(f_fhc)

In [ ]:
with open('variations/RHCVariations_July15.json') as f_rhc:
    rhc_dict = json.load(f_rhc)

## PPFX

In [ ]:
fig = plt.figure(figsize=(8, 5)) 

for v in fhc_dict['ppfx']: 
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=fhc_dict['evt_rate'],
        histtype='step', color='black', linewidth=2)    

plt.xlim(xlow, 2.5)
plt.title("PPFX VARIATIONS (FHC)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)

plt.show()

In [ ]:

fig = plt.figure(figsize=(8, 5)) 

for v in rhc_dict['ppfx']: 
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=rhc_dict['evt_rate'],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, 2.5)
plt.title("PPFX VARIATIONS (RHC)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
ppfx_variations = []

fig = plt.figure(figsize=(8, 5)) 

for v_fhc, v_rhc in zip(fhc_dict['ppfx'], rhc_dict['ppfx']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    ppfx_variations.append(comb_v)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, 2.5)
plt.title("PPFX VARIATIONS (FHC+RHC)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
importlib.reload(uncertainty_functions)
from uncertainty_functions import * 

In [ ]:
fhc_ppfx_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['ppfx'], plot=True, 
                    title='Hadronic Models', save=False) 

In [ ]:
rhc_ppfx_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['ppfx'], plot=True, 
                    title='Hadronic Models', save=False) 

In [ ]:
## calculate covariance 

ppfx_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    ppfx_variations, plot=True, 
                    title='Hadronic Models', save=False) 

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=ppfx_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_ppfx_dict['fractional_uncertainty'], linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_ppfx_dict['fractional_uncertainty'], linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.25)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("PPFX Uncertainty", fontsize=16)

plt.show()

## Beamline Geometry 

In [ ]:
beamline_runs = ['HornCurrent', 'xHorn1', 'yHorn1', 'BeamSpotSize', 'xHorn2', 'yHorn2', 'WaterOnHorns', 
                'xBeamShift', 'yBeamShift', 'zTargetPosition']

In [ ]:
len(fhc_dict['beamline'])

In [ ]:
fhc_beamline_unisim_cov = {}
rhc_beamline_unisim_cov = {}
beamline_unisim_cov = {}

i = 0

for v_fhc, v_rhc in zip(fhc_dict['beamline'], rhc_dict['beamline']): 
    
    print(beamline_runs[i])
    
    variations = []
    
    comb_v = [a+b for a,b in zip(v_fhc[0],v_rhc[0])]
    comb_v2 = [a+b for a,b in zip(v_fhc[1],v_rhc[1])]
    
    variations.append(comb_v)
    variations.append(comb_v2)
    
    fig = plt.figure(figsize=(8, 5)) 
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v2,
            histtype='step', color='cornflowerblue', linewidth=1)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
             weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
        histtype='step', color='black', linewidth=2)
              
    plt.xlim(xlow, 2.5)
    plt.title(beamline_runs[i]+" Variations (FHC+RHC)", fontsize=15)

    plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
    plt.yticks(fontsize=13)

    plt.xlabel(x_label, fontsize=14)
    plt.show()
    
    # compute unisim covariance
    
    fhc_beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                                                    v_fhc, plot=False, save=False)
    
    rhc_beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                                                    v_rhc, plot=False, save=False)
    
    beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                                    variations, plot=False, save=False)
    

    i += 1

In [ ]:
for k,d in enumerate([fhc_beamline_unisim_cov, rhc_beamline_unisim_cov, beamline_unisim_cov]): 
    
    cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    
    for variation in d.keys(): 

        for i in range(len(bins)-1): 
            for j in range(len(bins)-1):

                cov[i][j] += d[variation]['cov'][i][j]
                frac_cov[i][j] += d[variation]['frac_cov'][i][j] 


    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):

            if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                    cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
        
    if k==0 or k==1: 
        print(np.sqrt(np.diag(frac_cov)))
        
    else: 
        beamline_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 

        print(beamline_unisim_dict['fractional_uncertainty'])

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=beamline_unisim_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=[0.03565191, 0.02926658, 0.03848975, 0.04442238, 0.07051269, 0.06698694], 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=[0.03627028, 0.02598845, 0.03868571, 0.04285482, 0.04718215, 0.09892696], 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.15)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Beamline Uncertainty", fontsize=16)

plt.show()

## GENIE ms

In [ ]:
genie_variations = []

fig = plt.figure(figsize=(8, 5)) 

for v_fhc, v_rhc in zip(fhc_dict['genie_ms'], rhc_dict['genie_ms']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    genie_variations.append(comb_v)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, 2.5)
plt.title("GENIE VARIATIONS (FHC+RHC)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
fhc_genie_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['genie_ms'], plot=False) 

rhc_genie_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['genie_ms'], plot=False) 

In [ ]:
genie_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    genie_variations, plot=False, save=False) 

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=genie_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_genie_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_genie_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
#plt.ylim(0, 0.15)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GENIE (ms) Uncertainty", fontsize=16)

plt.show()

## GENIE us

In [ ]:
genie_unisims = ['RPA', 'RPA', 'CCMEC', 'AxFFCCQE', 'VecFFCCQE', 'DecayAngMEC', 'ThetaDelta2Npi', 'ThetaDelta2NRad', 
                          'NormCCCOH', 'NormNCCOH', 
                          'xsr_scc_Fv3', 'xsr_scc_Fa3']

In [ ]:
genie_unisim_cov = {}
fhc_genie_unisim_cov = {}
rhc_genie_unisim_cov = {}

i = 1

for v_fhc, v_rhc in zip(fhc_dict['genie_us'], rhc_dict['genie_us']): 
    
    print(genie_unisims[i])
    
    variations = []
    
    comb_v = [a+b for a,b in zip(v_fhc[0],v_rhc[0])]
    
    fig = plt.figure(figsize=(8, 5)) 
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)
    
    if i==1:
        comb_v2 = [a+b for a,b in zip(v_fhc[1],v_rhc[1])]
        #genie_us_variations.append([comb_v, comb_v2])
        variations.append(comb_v)
        variations.append(comb_v2)
    
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v2,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    else: 
        variations.append(comb_v)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
             weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
        histtype='step', color='black', linewidth=2)
              
    plt.xlim(xlow, 2.5)
    plt.title(genie_unisims[i]+" Variations (FHC+RHC)", fontsize=15)

    plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
    plt.yticks(fontsize=13)

    plt.xlabel(x_label, fontsize=14)
    plt.show()
    
    fhc_genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, 
                                                    fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                                                    v_fhc, plot=False, save=False)
    
    rhc_genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, 
                                                    rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                                                    v_rhc, plot=False, save=False)
    
    
    # compute unisim covariance
    genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                     [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                     variations, plot=False, save=False)
    

    i += 1
    

In [ ]:
for k,d in enumerate([fhc_genie_unisim_cov, rhc_genie_unisim_cov, genie_unisim_cov]): 
    
    cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    
    for variation in d.keys(): 

        for i in range(len(bins)-1): 
            for j in range(len(bins)-1):

                cov[i][j] += d[variation]['cov'][i][j]
                frac_cov[i][j] += d[variation]['frac_cov'][i][j] 


    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):

            if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                    cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
        
    if k==0 or k==1: 
        print(list(np.sqrt(np.diag(frac_cov))))
        
    else: 
        genie_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 

        print(genie_unisim_dict['fractional_uncertainty'])

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=genie_unisim_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=[0.07772277893028244, 0.023843003995427086, 0.02604231665943866, 0.021890860319963044, 0.03644975889572493, 0.03798103703037698], 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=[0.06853959481825295, 0.027662974240252797, 0.08520362055369854, 0.057560672201180545, 0.03285804162032622, 0.029561164059373767], 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.1)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GENIE (us) Uncertainty", fontsize=16)

plt.show()

## GEANT4

In [ ]:
geant4_variations = []

fig = plt.figure(figsize=(8, 5)) 

for v_fhc, v_rhc in zip(fhc_dict['geant4'], rhc_dict['geant4']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    geant4_variations.append(comb_v)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, 2.5)
plt.title("GEANT4 VARIATIONS (FHC+RHC)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
fhc_geant4_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['geant4'], plot=False) 

rhc_geant4_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['geant4'], plot=False) 

In [ ]:
geant4_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    geant4_variations, plot=False, save=False) 

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=geant4_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_geant4_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_geant4_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.05)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GEANT4 Uncertainty", fontsize=16)

plt.show()

## Detector systematics -- Using Reco X

In [ ]:
with open('variations/FHCDetSysVariations_July8.json') as f_fhc_detsys:
    fhc_detsys_dict = json.load(f_fhc_detsys)

In [ ]:
fhc_detsys_dict['LYAttenuation_intrinsic'] = [0 for i in range(len(fhc_detsys_dict['CV_intrinsic'] ))]

In [ ]:
with open('variations/RHCDetSysVariations_July8.json') as f_rhc_detsys:
    rhc_detsys_dict = json.load(f_rhc_detsys)

In [ ]:
xvar_detsys = "reco_nu_vtx_sce_x"
bins_detsys = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

In [ ]:
bincenters_detsys = 0.5*(np.array(bins_detsys)[1:]+np.array(bins_detsys)[:-1])

In [ ]:
x_err_detsys = [ round(abs(bins_detsys[x+1]-bins_detsys[x])/2, 3) for x in range(len(bins_detsys)-1) ]

In [ ]:
for variation in rhc_detsys_dict.keys(): 
    
    if variation=='CV_intrinsic': 
        continue
        
    plt.hist(bincenters_detsys, bins_detsys, histtype='step', range=[bins_detsys[0], bins_detsys[-1]], 
                 weights=[a+b for a,b in zip(fhc_detsys_dict[variation], rhc_detsys_dict[variation])], 
                            color='cornflowerblue', linewidth=0.5, label='UV')
    
        
    if variation=='LYAttenuation_intrinsic': 

        plt.errorbar(bincenters_detsys, 
                     rhc_detsys_dict['CV_intrinsic'], 
                     xerr=x_err_detsys, 
                     fmt='none', color='black', linewidth=2, label='CV (RHC only)')
        
    else: 

        plt.errorbar(bincenters_detsys, 
                     [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])], 
                     xerr=x_err_detsys, 
                     fmt='none', color='black', linewidth=2, label='CV')
    
    plt.xlabel(xvar, fontsize=14)
    plt.title(variation+' (nue CC events)')
    plt.legend()
    plt.show()

In [ ]:
detsys_cov_nueCC = {}

for variation in fhc_detsys_dict.keys(): 
    
    if variation=='CV': 
        continue
        
    if variation=='LYAttenuation_intrinsic': 
        
        detsys_cov_nueCC[variation] = calcCov(xvar, bins_detsys, 
                                          rhc_detsys_dict['CV_intrinsic'], 
                                              rhc_detsys_dict['CV_intrinsic'], 
                                              [rhc_detsys_dict[variation]])
        
    else: 
   
        detsys_cov_nueCC[variation] = calcCov(xvar_detsys, bins_detsys, 
                                          [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])], 
                                              [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])], 
                                              [[a+b for a,b in zip(fhc_detsys_dict[variation], rhc_detsys_dict[variation])]])
  

In [ ]:
cov = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]
frac_cov = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]

for i in range(len(bins_detsys)-1): 
    for j in range(len(bins_detsys)-1):
            
        cov[i][j] = sum([detsys_cov_nueCC[x]['cov'][i][j] for x in rhc_detsys_dict.keys() if x is not 'CV_intrinsic'])
        frac_cov[i][j] = sum([detsys_cov_nueCC[x]['frac_cov'][i][j] for x in rhc_detsys_dict.keys() if x is not 'CV_intrinsic'])

nueCC_detsys_dict = {
    'cov' : cov, 
    'frac_cov' : frac_cov,
    'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
} 

nueCC_detsys_dict['fractional_uncertainty']

In [ ]:

x_err = [ round(abs(bins[x+1]-bins[x])/2, 3) for x in range(len(bins)-1) ]

plt.figure(figsize=(8, 5))

cv_bdtcut = [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])]
cv_bdtcut_err = [np.sqrt(x)/x for x in cv_bdtcut]

plt.hist(bincenters_detsys, bins_detsys, weights=nueCC_detsys_dict['fractional_uncertainty'], histtype='step', 
         label='After BDT Cut (from 14712.2 GENIE/PPFX-tuned CV evts)', color='gray')

plt.errorbar(bincenters_detsys, nueCC_detsys_dict['fractional_uncertainty'], fmt='none', 
             yerr=cv_bdtcut_err, color='gray')


plt.hlines(np.average(nueCC_detsys_dict['fractional_uncertainty']), 0, 250, linestyle='--', 
           label='Avg = '+str(round(np.average(nueCC_detsys_dict['fractional_uncertainty']), 3)))

plt.xlim(0, 250)
plt.ylim(0, 0.33)
plt.legend(fontsize=13)

plt.xlabel("Reconstructed X Position [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("FHC Run 1 & RHC Run 3 Det. Sys. ($\\nu_{e}$ CC Intrinsic Sample)", fontsize=15)
#plt.savefig(parameters(ISRUN3)['plots_path']+"RHCRUN3DetSys.pdf", transparent=True, bbox_inches='tight') 

plt.show()

In [ ]:
avg_detsys_dict = {
    'fractional_uncertainty' : [np.average(nueCC_detsys_dict['fractional_uncertainty']) for x in range(len(bins))]
}

In [ ]:
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for i in range(len(bins)-1): 
        frac_cov[i][i] = np.average(nueCC_detsys_dict['fractional_uncertainty'])**2

avg_detsys_dict['frac_cov'] = frac_cov

## Stat Uncertainty (MC background) 

In [ ]:
# doesn't include EXT uncertainty 
# sum of the weights squared

mc_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
mc_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

mc_sumw2 = [a+b for a,b in zip(fhc_dict['mc_bkgd_sumw2'], rhc_dict['mc_bkgd_sumw2'])]
evt_rate = [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])]

fhc_mc_stat_percent_error = []
rhc_mc_stat_percent_error = []

# MC background stat uncertainty
for i in range(len(bins)-1):
    
    mc_stat_cov[i][i] = mc_sumw2[i]
    mc_frac_stat_cov[i][i] = mc_stat_cov[i][i]/ evt_rate[i]**2 
    
    fhc_mc_stat_percent_error.append(np.sqrt(fhc_dict['mc_bkgd_sumw2'][i]/fhc_dict['evt_rate'][i]**2))
    rhc_mc_stat_percent_error.append(np.sqrt(rhc_dict['mc_bkgd_sumw2'][i]/rhc_dict['evt_rate'][i]**2))

    
mc_stat_percent_error = np.sqrt(np.diag(mc_frac_stat_cov))
mc_stat_percent_error

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=mc_stat_percent_error, color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_mc_stat_percent_error, 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_mc_stat_percent_error, 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
#plt.ylim(0, 0.05)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("MC Bkgd STAT Uncertainty", fontsize=16)

plt.show()


## Stat Uncertainty (EXT)

In [ ]:
# EXT uncertainty 
# sum of the weights squared

ext_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
ext_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

ext_sumw2 = [a+b for a,b in zip(fhc_dict['ext_sumw2'], rhc_dict['ext_sumw2'])]

fhc_ext_stat_percent_error = []
rhc_ext_stat_percent_error = []

# MC background stat uncertainty
for i in range(len(bins)-1):
    
    ext_stat_cov[i][i] = ext_sumw2[i]
    ext_frac_stat_cov[i][i] = ext_stat_cov[i][i]/ evt_rate[i]**2 
    
    fhc_ext_stat_percent_error.append(np.sqrt(fhc_dict['ext_sumw2'][i]/fhc_dict['evt_rate'][i]**2))
    rhc_ext_stat_percent_error.append(np.sqrt(rhc_dict['ext_sumw2'][i]/rhc_dict['evt_rate'][i]**2))
    
ext_stat_percent_error = np.sqrt(np.diag(ext_frac_stat_cov))
ext_stat_percent_error

In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=ext_stat_percent_error, color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_ext_stat_percent_error, 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_ext_stat_percent_error, 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
#plt.ylim(0, 0.05)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("EXT STAT Uncertainty", fontsize=16)

plt.show()

## Stat Uncertainty (BEAM ON)

In [ ]:
# background subtracted event rate (use MC for closure test/fake data studies, beam ON count for real data result)

print("Make sure to update to beam on statistics when using real data !")
    
beamon_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

fhc_beamon_stat_percent_error = []
rhc_beamon_stat_percent_error = []
    
for i in range(len(bins)-1): 
    
    beamon_frac_stat_cov[i][i] = evt_rate[i]/(evt_rate[i]**2)
    
    fhc_beamon_stat_percent_error.append(np.sqrt(fhc_dict['evt_rate'][i]/fhc_dict['evt_rate'][i]**2))
    rhc_beamon_stat_percent_error.append(np.sqrt(rhc_dict['evt_rate'][i]/rhc_dict['evt_rate'][i]**2))

beamon_stat_percent_error = np.sqrt(np.diag(beamon_frac_stat_cov))
print(beamon_stat_percent_error) 

In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=beamon_stat_percent_error, color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_beamon_stat_percent_error, 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_beamon_stat_percent_error, 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.65)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Beam-On Uncertainty", fontsize=16)

plt.show()

## Response Matrix 

In [ ]:
rmatrix_variations = []

fig = plt.figure(figsize=(8, 5)) 

for v_fhc, v_rhc in zip(fhc_dict['response_matrix'], rhc_dict['response_matrix']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    rmatrix_variations.append(comb_v)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, 2.5)
plt.title("RESPONSE MATRIX VARIATIONS (FHC+RHC)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
fhc_rmatrix_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['response_matrix'], plot=False, save=False) 

rhc_rmatrix_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['response_matrix'], plot=False, save=False) 

In [ ]:
rmatrix_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    rmatrix_variations, plot=False, save=False) 

In [ ]:
rmatrix_dict['fractional_uncertainty']

In [ ]:
fig = plt.figure(figsize=(8, 5))  

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=rmatrix_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_rmatrix_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_rmatrix_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.1)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Response Matrix Uncertainty", fontsize=16)

plt.show()

## Dirt Uncertainty 

In [ ]:
importlib.reload(uncertainty_functions)
from uncertainty_functions import * 

In [ ]:
dirt_variations = []

fig = plt.figure(figsize=(8, 5)) 


comb_v = [a+b for a,b in zip(fhc_dict['dirt'],rhc_dict['dirt'])]
dirt_variations.append(comb_v)
    
plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['cv_dirt'], rhc_dict['cv_dirt'])],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, 2.5)
plt.title("DIRT VARIATIONS (FHC+RHC)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
fhc_dirt_dict = dirt_unisim(xvar, bins, fhc_dict['evt_rate'], fhc_dict['cv_dirt'], 1.0, isrun3=False, plot=True, 
                               x_label=None, title=None)

rhc_dirt_dict = dirt_unisim(xvar, bins, rhc_dict['evt_rate'], rhc_dict['cv_dirt'], 1.0, isrun3=True, plot=True, 
                               x_label=None, title=None)

In [ ]:
dirt_dict = dirt_unisim(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                        [a+b for a,b in zip(fhc_dict['cv_dirt'], rhc_dict['cv_dirt'])], 
                        1.0, plot=True, x_label=None, title=None)

In [ ]:
fig = plt.figure(figsize=(8, 5))  

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=dirt_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_dirt_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_dirt_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='orange', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
#plt.ylim(0, 0.1)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Dirt Uncertainty", fontsize=16)

plt.show()

## POT counting 

In [ ]:
pot_variations = []

fig = plt.figure(figsize=(8, 5)) 

for v_fhc, v_rhc in zip(fhc_dict['pot_counting'], rhc_dict['pot_counting']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    pot_variations.append(comb_v)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, 2.5)
plt.title("POT COUNTING VARIATIONS (FHC+RHC)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
pot_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    pot_variations, plot=False, save=False) 

In [ ]:
pot_dict['fractional_uncertainty']

## Full Uncertainty 

In [ ]:
frac_cov_dict = {
    'ppfx' : ppfx_dict['frac_cov'], 
    'beamline' : beamline_unisim_dict['frac_cov'], 
    'genie_ms' : genie_dict['frac_cov'], 
    'genie_us': genie_unisim_dict['frac_cov'], 
    'geant4' : geant4_dict['frac_cov'],
    'detector' : avg_detsys_dict['frac_cov'], 
    'pot_counting' : pot_dict['frac_cov'], 
    'dirt' : dirt_dict['frac_cov'],
    'mc_stat' : mc_frac_stat_cov, # background only 
    'ext_stat' : ext_frac_stat_cov, 
    'response_matrix' : rmatrix_dict['frac_cov']
}

In [ ]:
tot_frac_cov, tot_abs_cov = plotFullCov(frac_cov_dict, xvar, evt_rate, bins, xlow, xhigh, x_ticks=x_ticks, save=False, 
                      axis_label='Reco '+x_label)

In [ ]:
# add ppfx & beamline geometry in quadrature
frac_cov_dict['flux'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['ppfx'], frac_cov_dict['beamline'])]

# add genie in quadrature
frac_cov_dict['genie_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['genie_ms'], frac_cov_dict['genie_us'])]

In [ ]:
print("MAKE sure to update to include beam on stat!")
frac_cov_dict['stat_all'] = [ [x+y+z for x,y,z in zip(b,c,d)] for b,c,d in zip( frac_cov_dict['response_matrix'], frac_cov_dict['mc_stat'], frac_cov_dict['ext_stat'])]


In [ ]:
# clean away nans
v = np.array(tot_frac_cov)
v[np.isnan(v)] = 0
tot_frac_cov = v

In [ ]:
frac_unc_dict = {
    'flux' : np.sqrt(np.diagonal(frac_cov_dict['flux'])), 
    'genie' : np.sqrt(np.diagonal(frac_cov_dict['genie_all'])), 
    'geant4' : np.sqrt(np.diagonal(frac_cov_dict['geant4'])),
    'detector' : np.sqrt(np.diagonal(frac_cov_dict['detector'])), 
    'pot_counting' : np.sqrt(np.diagonal(frac_cov_dict['pot_counting'])), 
    'dirt' : np.sqrt(np.diagonal(frac_cov_dict['dirt'])),
    'stat' : np.sqrt(np.diagonal(frac_cov_dict['stat_all'])), # does not include beam on STAT 
    'total' : np.sqrt(np.diagonal(tot_frac_cov))
}

In [ ]:
tot_unc = [0 for i in range(len(bins)-1)]
test = []

for source in frac_unc_dict.keys(): 
    
    if source=='total': 
        continue
    
    test.append(frac_unc_dict[source][0])
    
    # square the list 
    squared = [x**2 for x in frac_unc_dict[source]]
    
    # add in quadrature 
    tot_unc = [a+b for a,b in zip(tot_unc, squared)]
   
tot_unc = np.sqrt(np.array(tot_unc))
print(tot_unc)

frac_unc_dict['total'] = tot_unc

In [ ]:


fig = plt.figure(figsize=(8, 5))  

# TOTAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Total",
        weights=frac_unc_dict['total'], linewidth=1.5, color='black')

# FLUX
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Flux", 
         weights=frac_unc_dict['flux'], color='royalblue')

# CROSS SECTION MODELS 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GENIE", 
         weights=frac_unc_dict['genie'], color='goldenrod')

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GEANT4", 
         weights=frac_unc_dict['geant4'], color='green')

# DETECTOR 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Detector", 
         weights=frac_unc_dict['detector'], color='crimson')

# POT COUNTING 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="POT counting",
        weights=frac_unc_dict['pot_counting'], color='purple')

# DIRT 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Dirt",
        weights=frac_unc_dict['dirt'], color='brown')

# STATISTICAL 

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Response matrix",
        weights=np.sqrt(np.diagonal(frac_cov_dict['response_matrix'])))

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Stat (MC+EXT)",
        weights=frac_unc_dict['stat'], color='hotpink')

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.5)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("FHC+RHC Background-Subtracted Event Rate", fontsize=16)

#plt.savefig(plots_path+xvar+"_RHC_FracUncertainty.pdf", transparent=True, bbox_inches='tight')

plt.show()

## Smearing & efficiency 

In [ ]:
with open('smearing/FHC_July15.json') as f_fhc:
    fhc_smearing_dict = json.load(f_fhc)

In [ ]:
with open('smearing/RHC_July15.json') as f_rhc:
    rhc_smearing_dict = json.load(f_rhc)

In [ ]:
fhc_smearing_dict.keys()

In [ ]:
binwidth = []

for x in range(len(bincenters)): 
    binwidth.append(round(abs(x_ticks[x+1]-x_ticks[x])/2, 2))
    
binwidth

In [ ]:
###########################################################################################

# fhc efficiency 
fhc_true_generated = fhc_smearing_dict['true_generated']
fhc_true_selected = fhc_smearing_dict['true_selected']

fhc_eff = [a/b for a,b in zip(fhc_true_selected, fhc_true_generated)]
fhc_eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(fhc_eff,fhc_true_generated) ]

print('FHC Efficiency (%) = ', [round(x,3)*100 for x in fhc_eff])

###########################################################################################

# rhc efficiency 
rhc_true_generated = rhc_smearing_dict['true_generated']
rhc_true_selected = rhc_smearing_dict['true_selected']

rhc_eff = [a/b for a,b in zip(rhc_true_selected, rhc_true_generated)]
rhc_eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(rhc_eff,rhc_true_generated) ]

print('RHC Efficiency (%) = ', [round(x,3)*100 for x in rhc_eff])

###########################################################################################

# combined efficiency 

true_generated = [a+b for a,b in zip(fhc_smearing_dict['true_generated'], rhc_smearing_dict['true_generated'])]
true_selected = [a+b for a,b in zip(fhc_smearing_dict['true_selected'], rhc_smearing_dict['true_selected'])]

eff = [a/b for a,b in zip(true_selected, true_generated)]
eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(eff,true_generated) ]

print('Combined Efficiency (%) = ', [round(x,3)*100 for x in eff])


# Plot 

fig = plt.figure(figsize=(8, 5))  

plt.errorbar(bincenters, fhc_eff, yerr=fhc_eff_err, xerr=binwidth, label="FHC", fmt='.')
plt.errorbar(bincenters, rhc_eff, yerr=rhc_eff_err, xerr=binwidth, label="RHC", fmt='.')

plt.errorbar(bincenters, eff, yerr=eff_err, xerr=binwidth, label="FHC+RHC", fmt='.', color='black')

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.ylim(0, .25)

plt.xlabel("True " + x_label, fontsize=15)
plt.ylabel("Selected / Generated", fontsize=15)

plt.xlim(bins[0], xhigh)

plt.legend(fontsize=14, frameon=False, ncol=3)
plt.title("Efficiency", fontsize=16)

plt.show()

In [ ]:
response_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for col in range(len(bins)-1):
    for row in range(len(bins)-1):

        response_matrix[row][col] = (fhc_smearing_dict['smearing_matrix'][row][col] + rhc_smearing_dict['smearing_matrix'][row][col]) / true_generated[col]



In [ ]:
fig = plt.figure(figsize=(13, 9))

plt.pcolor(bins, bins, response_matrix, cmap='OrRd', vmin=0.00000000001, edgecolors='k')

plt.title('FHC+RHC Smearing & Efficiency', fontsize=15)

plt.xticks(x_ticks, fontsize=14)
plt.gca().xaxis.tick_bottom()
plt.yticks(x_ticks, fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(xlow, xhigh)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xlabel('True Shower Energy [GeV]', fontsize=15)
plt.ylabel('Reco Shower Energy [GeV]', fontsize=15)

plt.show()

In [ ]:
# check that this is constructed properly 
# sum of the columns should equal the combined efficiency 

for i in range(len(bincenters)): # loop over columns

    x = 0 
    for j in range(len(bincenters)): # sum up the rows
    
        x += response_matrix[j][i]

    print(x)

In [ ]:
eff

## CV event rate

In [ ]:
fig = plt.figure(figsize=(10, 7))


plt.hist(bincenters, bins, color='orange', label='True (Generated)', alpha=0.4, 
         weights=[a+b for a,b in zip(fhc_smearing_dict['true_generated'],rhc_smearing_dict['true_generated'])])

plt.hist(bincenters, bins, color='blue', label='Reco (Selected)', alpha=0.4, weights=evt_rate)

plt.legend(fontsize=13)
plt.title('FHC+RHC Input Signal Distributions', fontsize=16)

#plt.ylim(0, 5E-40)
plt.xlim(0, 2.5)

plt.xlabel("Shower Energy [GeV]", fontsize=15)
    
plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

#plt.savefig(plots_path+"_InputSignalDistributions_ElectronEnergy.pdf", transparent=True, bbox_inches='tight') 

plt.grid()
plt.show()

## save to unfolding file 

In [ ]:
save = False

if save: 
    print("Make sure to update file name!")
    

In [ ]:
if save: 
    
    # covariance matrix 
    hcov = TH2D("hcov_tot", "Covariance Matrix vs. Reco "+x_label, 
                len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))

    for i in range(len(bincenters)): # i = row (y)
        for j in range(len(bincenters)): # j = column (x) 

            hcov.Fill(bincenters[j], bincenters[i], tot_abs_cov[i][j]) 

    # true signal distribution
    htrue_signal = TH1F("htrue_signal", "Generated MC Signal vs. True "+x_label+" ("+true_var+")", len(bins)-1, np.array(bins))

    for i in range(len(bincenters)): 
        htrue_signal.Fill(bincenters[i], true_generated[i])
        
    # reco distribution (MC or data)
    hreco = TH1D("hmeas", "Selected MC Signal vs. Reco "+x_label+" ("+xvar+")", len(bins)-1, np.array(bins))

    for i in range(len(bincenters)): 
        hreco.Fill(bincenters[i], evt_rate[i])
    
    # response matrix 
    r = TH2D("hR", "Response Matrix", len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))
    for i in range(len(bincenters)): # i = column (x)
        for j in range(len(bincenters)): # j = row (y) 
            
            r.Fill(bincenters[i], bincenters[j], response_matrix[i][j]) 
            
    f = ROOT.TFile.Open("/uboone/data/users/kmiller/uBNuMI_CCNp/unfolding/WSVD_"+xvar+"_Combined_July15.root", "RECREATE")
    
    f.cd()
    
    hcov.Write()
    htrue_signal.Write()
    hreco.Write()
    r.Write()
    
    f.Close()

save = False